<a href="https://colab.research.google.com/github/kashifalikhan36/Machine_learning_Resources/blob/main/Time_Series_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
NUM_OF_PREV_ITEMS=5
df=pd.read_csv("daily-min-temperatures.csv")
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [8]:
np.random.seed(1)

In [9]:
data=df[["Temp"]].values
data=data.astype('float32')
scaler=MinMaxScaler(feature_range=(0,1))
data=scaler.fit_transform(data)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, data, test_size=0.2, random_state=42)

In [11]:
#Reconstruct the data as that they will be in 1 columns and having ll the 5 values and the target will have 5th value
def reconstruct_data(data, n_prev = 1):
    docX, docY = [], []
    for i in range(len(data)-n_prev-1):
        docX.append(data[i:(i+n_prev)])
        docY.append(data[(i+n_prev)])
    return np.array(docX), np.array(docY)

In [12]:
train_x, train_y = reconstruct_data(X_train, NUM_OF_PREV_ITEMS)
test_x, test_y = reconstruct_data(X_test, NUM_OF_PREV_ITEMS)

In [22]:
print(train_x[0][0]) #Features
print(train_y[0]) #Target

[0.6387833  0.38403046 0.6768061  0.5285171  0.2889734 ]
0.38783273


In [13]:
#Reshape to [numOfSample,time steps, numOfFeatures]
#step is 1 because we want to predict the next value (t+1)
train_x=np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
test_x=np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))

In [15]:
model=Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(1, NUM_OF_PREV_ITEMS)))
model.add(Dropout(0.6))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(50))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=50, batch_size=16,verbose=2)

Epoch 1/50
183/183 - 7s - 39ms/step - loss: 0.0468
Epoch 2/50
183/183 - 1s - 4ms/step - loss: 0.0261
Epoch 3/50
183/183 - 1s - 7ms/step - loss: 0.0262
Epoch 4/50
183/183 - 1s - 7ms/step - loss: 0.0256
Epoch 5/50
183/183 - 1s - 4ms/step - loss: 0.0249
Epoch 6/50
183/183 - 1s - 7ms/step - loss: 0.0253
Epoch 7/50
183/183 - 1s - 4ms/step - loss: 0.0249
Epoch 8/50
183/183 - 2s - 13ms/step - loss: 0.0252
Epoch 9/50
183/183 - 2s - 9ms/step - loss: 0.0248
Epoch 10/50
183/183 - 2s - 10ms/step - loss: 0.0246
Epoch 11/50
183/183 - 1s - 7ms/step - loss: 0.0246
Epoch 12/50
183/183 - 1s - 4ms/step - loss: 0.0247
Epoch 13/50
183/183 - 1s - 7ms/step - loss: 0.0245
Epoch 14/50
183/183 - 1s - 4ms/step - loss: 0.0245
Epoch 15/50
183/183 - 1s - 4ms/step - loss: 0.0243
Epoch 16/50
183/183 - 1s - 6ms/step - loss: 0.0242
Epoch 17/50
183/183 - 1s - 7ms/step - loss: 0.0243
Epoch 18/50
183/183 - 1s - 8ms/step - loss: 0.0242
Epoch 19/50
183/183 - 2s - 11ms/step - loss: 0.0243
Epoch 20/50
183/183 - 2s - 9ms/step 

In [19]:
test_pred=model.predict(test_x)
mean_squared_error(test_pred, test_y)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


0.024820532649755478